# Sports14 Text/Image Feature Extraction

In [1]:

import os
import numpy as np
import pandas as pd

In [2]:
dataset = 'toys'
rslt_dir = f'/home/yueqi/peilin/MMRec/data/AmazonReview/{dataset}/'
image_dir = f'/home/yueqi/peilin/MMRec/AmazonRawData/Amazon_Review_Plus/photos/{dataset}/'
image_features = f'image_features_{dataset}.b'
os.chdir(rslt_dir)
os.getcwd()
i_id_mapping = 'i_id_mapping.csv'
meta_data_path = rslt_dir + f'meta-{dataset}.csv'

## Load text data

In [3]:
i_id, desc_str = 'itemID', 'description'


df = pd.read_csv(meta_data_path)
df.sort_values(by=[i_id], inplace=True)

print('data loaded!')
print(f'shape: {df.shape}')

df[:3]

data loaded!
shape: (11924, 10)


,itemID,asin,title,price,imUrl,related,brand,categories,salesRank,description
4066,0,B001RNFQNK,Hasbro Electronic Catch Phrase,29.99,413BcULsveL._SX300_.jpg,"{'also_bought': ['B00H4OKN48', 'B001RNC0VG', '...",Parker Brothers,"['Toys & Games', 'Games', 'Handheld Games']",{'Toys & Games': 3095},Classic Electronic Catch Phrase with all new p...
64,1,158978068X,Gloom,17.28,51H3uVu1VbL._SX300_.jpg,{},Atlas,"['Toys & Games', 'Games', 'Board Games']",NaN,NaN
7248,2,B004S8F7QM,Cards Against Humanity,25.00,41164wOO6ZL._SX300_.jpg,"{'also_bought': ['B005JFNE8G', 'B008JNPBYK', '...",NaN,"['Toys & Games', 'Games', 'Card Games']",{'Toys & Games': 1},Cards Against Humanity is a party game for hor...


In [4]:

# sentences: title + brand + category + description | All have title + description

title_na_df = df[df['title'].isnull()]
print(title_na_df.shape)

desc_na_df = df[df['description'].isnull()]
print(desc_na_df.shape)

na_df = df[df['description'].isnull() & df['title'].isnull()]
print(na_df.shape)

na3_df = df[df['description'].isnull() & df['title'].isnull() & df['brand'].isnull()]
print(na3_df.shape)

na4_df = df[df['description'].isnull() & df['title'].isnull() & df['brand'].isnull() & df['categories'].isnull()]
print(na4_df.shape)

(59, 10)
(798, 10)
(34, 10)
(34, 10)
(0, 10)


In [5]:

df[desc_str] = df[desc_str].fillna(" ")
df['title'] = df['title'].fillna(" ")
df['brand'] = df['brand'].fillna(" ")
df['categories'] = df['categories'].fillna(" ")


In [6]:
sentences = []
for i, row in df.iterrows():
    sen = row['title'] + ' ' + row['brand'] + ' '
    cates = eval(row['categories'])
    if isinstance(cates, list):
        for c in cates[0]:
            sen = sen + c + ' '
    sen += row[desc_str]
    sen = sen.replace('\n', ' ')

    sentences.append(sen)

sentences[:10]

['Hasbro Electronic Catch Phrase Parker Brothers T o y s   &   G a m e s Classic Electronic Catch Phrase with all new phrases to play. Say it fast, play it fast, it\'s a blast. How quickly can you get your teammates to say "backseat driver"? "Pay through the nose"? "Strong, silent type"? You can do or say just about anything to get them to guess the phrase',
 'Gloom Atlas T o y s   &   G a m e s  ',
 'Cards Against Humanity   T o y s   &   G a m e s Cards Against Humanity is a party game for horrible people.Unlike most of the party games you\'ve played before, Cards Against Humanity is as despicable and awkward as you and your friends.The game is simple. Each round, one player asks a question from a Black Card, and everyone else answers with their funniest White Card.Reviews of Cards Against Humanity:"Pretty amazing." -The Onion AV Club"An incredible game." -Mike "Gabe" Krahulik, Penny Arcade"Uncontrollable laughter." -Kill Screen Magazine"The game your party deserves." -Thrillist"A ga

In [7]:

course_list = df[i_id].apply(int).tolist()
#sentences = df[desc_str].tolist()
lastID = course_list[-1]
lastIdx = len(course_list) - 1
print(f"lastID: {lastID} lastIdx: {lastIdx}")
assert course_list[-1] == len(course_list) - 1

lastID: 11923 lastIdx: 11923


In [8]:
# should `pip install sentence_transformers` first
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('all-MiniLM-L6-v2')

sentence_embeddings = model.encode(sentences)
print('text encoded!')

assert sentence_embeddings.shape[0] == df.shape[0]
np.save(os.path.join(rslt_dir, 'text_feat.npy'), sentence_embeddings)
print('done!')


/home/yueqi/anaconda3/envs/MMRec/lib/python3.7/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


text encoded!
done!


In [9]:
sentence_embeddings[:10]

array([[-0.01760189, -0.01881865,  0.0114401 , ...,  0.05846089,
        -0.00442534, -0.02347583],
       [-0.03766672,  0.06037303,  0.03006689, ..., -0.04724079,
        -0.01372679, -0.08072826],
       [-0.06160678,  0.05235229, -0.05429302, ..., -0.00620484,
        -0.06693427,  0.0084694 ],
       ...,
       [ 0.03095264, -0.00148614,  0.0436652 , ...,  0.01172378,
         0.01494396, -0.01617399],
       [-0.09353577,  0.02738318,  0.03217479, ..., -0.04501066,
         0.0366418 , -0.0262657 ],
       [ 0.05711336, -0.02056006,  0.05397408, ...,  0.05541417,
         0.03956362,  0.00096201]], dtype=float32)

In [10]:
load_txt_feat = np.load('text_feat.npy', allow_pickle=True)
print(load_txt_feat.shape)
load_txt_feat[:10]

(11924, 384)


array([[-0.01760189, -0.01881865,  0.0114401 , ...,  0.05846089,
        -0.00442534, -0.02347583],
       [-0.03766672,  0.06037303,  0.03006689, ..., -0.04724079,
        -0.01372679, -0.08072826],
       [-0.06160678,  0.05235229, -0.05429302, ..., -0.00620484,
        -0.06693427,  0.0084694 ],
       ...,
       [ 0.03095264, -0.00148614,  0.0436652 , ...,  0.01172378,
         0.01494396, -0.01617399],
       [-0.09353577,  0.02738318,  0.03217479, ..., -0.04501066,
         0.0366418 , -0.0262657 ],
       [ 0.05711336, -0.02056006,  0.05397408, ...,  0.05541417,
         0.03956362,  0.00096201]], dtype=float32)

# Image encoder (V0)，following LATTICE, averaging over for missed items

In [11]:
df[:5]

,itemID,asin,title,price,imUrl,related,brand,categories,salesRank,description
4066,0,B001RNFQNK,Hasbro Electronic Catch Phrase,29.99,413BcULsveL._SX300_.jpg,"{'also_bought': ['B00H4OKN48', 'B001RNC0VG', '...",Parker Brothers,"['Toys & Games', 'Games', 'Handheld Games']",{'Toys & Games': 3095},Classic Electronic Catch Phrase with all new p...
64,1,158978068X,Gloom,17.28,51H3uVu1VbL._SX300_.jpg,{},Atlas,"['Toys & Games', 'Games', 'Board Games']",NaN,
7248,2,B004S8F7QM,Cards Against Humanity,25.00,41164wOO6ZL._SX300_.jpg,"{'also_bought': ['B005JFNE8G', 'B008JNPBYK', '...",,"['Toys & Games', 'Games', 'Card Games']",{'Toys & Games': 1},Cards Against Humanity is a party game for hor...
9779,3,B008ULAMSG,Carcassonne Basic Game,25.86,51286n8bV6L._SY300_.jpg,"{'also_bought': ['B00BJLUGNK', 'B00008URUN', '...",Z-Man Games,"['Toys & Games', 'Games', 'Board Games']",{'Toys & Games': 679},The Carcassonne is a clever tile-laying game. ...
10789,4,B00BYPG57W,Asmodee 7 Wonders Wonder Pack,11.90,51uIPYBgHcL._SX300_.jpg,"{'also_bought': ['B0055CAF6M', 'B007IW0HME', '...",Asmodee,"['Toys & Games', 'Games', 'Board Games']",{'Toys & Games': 3824},"In 7 Wonders Wonder Pack, players will obtain ..."


In [12]:
import array

def readImageFeatures(path):
  f = open(path, 'rb')
  while True:
    asin = f.read(10).decode('UTF-8')
    if asin == '': break
    a = array.array('f')
    a.fromfile(f, 4096)
    yield asin, a.tolist()

In [13]:

img_data = readImageFeatures(image_features)
item2id = dict(zip(df['asin'], df['itemID']))

feats = {}
avg = []
for d in img_data:
    if d[0] in item2id:
        feats[int(item2id[d[0]])] = d[1]
        avg.append(d[1])
avg = np.array(avg).mean(0).tolist()

ret = []
non_no = []
for i in range(len(item2id)):
    if i in feats:
        ret.append(feats[i])
    else:
        non_no.append(i)
        ret.append(avg)

print('# of items not in processed image features:', len(non_no))
assert len(ret) == len(item2id)
np.save('image_feat.npy', np.array(ret))
np.savetxt("missed_img_itemIDs.csv", non_no, delimiter =",", fmt ='%d')
print('done!')

# of items not in processed image features: 86
done!
